In [3]:
#NOTE: MUST RUN THIS FROM MAIN DIRECTORY ELSE STYLE.CSS DOESN'T WORK

import pandas as pd
import dash
from dash import dash_table, html
import numpy as np



def generate_main_table():
    df = pd.read_csv('./data/wps_gte_2015_pivot.csv',parse_dates=['period'])

    df['period'] = df['period'].dt.strftime('%m/%d')
    df = df.tail(2)
    df = df.set_index('period')
    df = df.T.reset_index()
    df['change'] = df.iloc[:,2] - df.iloc[:,1]
    df = df.rename(columns={'index':'name'})

    df.columns.name = None

    return df
df  = generate_main_table()

In [4]:


def generate_dash_table(df, idents, table_id, table_width):
    df = df.copy()

    def format_number(num):
        # Force all to numeric
        num = pd.to_numeric(num, errors='coerce')
        num = round(num, 1)
        return num
    
    col_first = df.columns[0]
    col_second = df.columns[1]
    col_third = df.columns[2]
    col_fourth = df.columns[3]
    cols = [col_second, col_third, col_fourth]
    
    for col in cols:
        df[col] = df[col].apply(format_number)
    
    idents_ids = list(idents.keys())
    idents_names = list(idents.values())
    df = df[df['name'].isin(idents_ids)]

    df = df.set_index('name')
    df = df.reindex(idents_ids)
    df = df.reset_index()
    
    df['name'] = df['name'].replace(idents_ids, idents_names)
    
    df = df.rename(columns={'name': table_id})
    df = df.reset_index(drop=True)

    col_first = table_id

    return html.Div(
        dash_table.DataTable(
            id=table_id,
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            style_table={
                'width': table_width, 
                'border': 'none', 
                'borderRadius': '15px', 
                # 'overflow': 'hidden', 
                'boxShadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)'
            },
            style_cell={
                'textAlign': 'left',
                'padding': '5px',
                'fontFamily': 'Arial',
                'fontSize': '10px',
                'border': '1px solid lightgrey',
                'borderRadius': '10px'
            },
            style_header={
                'backgroundColor': '#bfbec4',
                'color': 'black',
                'fontWeight': 'bold'
            },
            style_cell_conditional=[
                {'if': {'column_id': col_first}, 'width': '270px'},
                {'if': {'column_id': col_second}, 'width': '65px', 'textAlign': 'right'},
                {'if': {'column_id': col_third}, 'width': '65px', 'textAlign': 'right'},
                {'if': {'column_id': col_fourth}, 'width': '65px', 'textAlign': 'right'},
            ],
            style_data_conditional=[
                {
                    'if': {
                        'filter_query': f'{{{col_fourth}}} > 0',
                        'column_id': col_fourth
                    },
                    'backgroundColor': 'lightgreen',
                    'color': 'black',
                },
                {
                    'if': {
                        'filter_query': f'{{{col_fourth}}} < 0',
                        'column_id': col_fourth
                    },
                    'backgroundColor': 'lightpink',
                    'color': 'black',
                }
            ]
        ),
        style={'margin-bottom': '20px'}
    )


In [5]:
headline = {
    'WCESTUS1' : 'US Commercial Stocks (kb)',
    'W_EPC0_SAX_YCUOK_MBBL' : 'Cushing Stocks (kb)',    
    'WGTSTUS1' : 'US Gasoline Stocks (kb)', 
    'WDISTUS1' : 'US Distillate Stocks (kb)',       
}

products_supplied = {
    'WRPUPUS2' : 'Products (kbd)',
    'WGFUPUS2' : 'Gasoline (kbd)',
    'WDIUPUS2' : 'Distillate (kbd)',
    'WKJUPUS2' : 'Jet (kbd)',
    'WREUPUS2' : 'Fuel Oil (kbd)',
    'WPRUP_NUS_2' : ' C3/C3= (kbd)',
    'WWOUP_NUS_2' : 'Other Oils (kbd)',
}

crude_stocks = {
    'WCESTUS1' : 'US Commercial Stocks (kb)',
    'WCESTP11' : 'P1 Commercial Stocks (kb)',
    'WCESTP21' : 'P2 Commercial Stocks (kb)',
    'WCESTP31' : 'P3 Commercial Stocks (kb)',
    'WCESTP41' : 'P4 Commercial Stocks (kb)',
    'WCESTP51' : 'P5 Commercial Stocks (kb)',    
}

crude_other_stocks = {
    'W_EPC0_SAX_YCUOK_MBBL' : 'Cushing Stocks (kb)',
    'crudeStocksP2E' : 'P2E Stocks (kb)',
    'WCSSTUS1' : 'SPR Stocks (kb)',
    'W_EPC0_SKA_NUS_MBBL' : 'Alaska Stocks (kb)',
    'WCRSTUS1' : 'Total Stocks (kb)',    
}

crude_production = {
    'WCRFPUS2': 'US Production (kbd)',
    'W_EPC0_FPF_R48_MBBLD': 'L48 Production (kbd)',
    'W_EPC0_FPF_SAK_MBBLD': 'AK Production (kbd)',            
}

crude_imports = {
    'WCEIMUS2' : 'US Imports (kbd)',
    'WCEIMP12' : 'P1 Imports (kbd)',
    'WCEIMP22' : 'P2 Imports (kbd)',
    'WCEIMP32' : 'P3 Imports (kbd)',
    'WCEIMP42' : 'P4 Imports (kbd)',
    'WCEIMP52' : 'P5 Imports (kbd)',    
}

crude_adjustment = {
    'crudeOriginalAdjustment' : 'OG Adjustment Factor (kbd)',
}

crude_runs = {
    'WCRRIUS2' : 'US Refinery Runs (kbd)',
    'WCRRIP12' : 'P1 Refinery Runs (kbd)',
    'WCRRIP22' : 'P2 Refinery Runs (kbd)',
    'WCRRIP32' : 'P3 Refinery Runs (kbd)',
    'WCRRIP42' : 'P4 Refinery Runs (kbd)',
    'WCRRIP52' : 'P5 Refinery Runs (kbd)',    
}

crude_exports = {
    'WCREXUS2' : 'US Crude Exports (kbd)',    
}

gasoline_stocks = {
    'WGTSTUS1' : 'US Gasoline Stocks (kb)',
    'WGTSTP11' : 'P1 Gasoline Stocks (kb)',
    'WGTSTP21' : 'P2 Gasoline Stocks (kb)',
    'WGTSTP31' : 'P3 Gasoline Stocks (kb)',
    'WGTSTP41' : 'P4 Gasoline Stocks (kb)',
    'WGTSTP51' : 'P5 Gasoline Stocks (kb)',    
}

gasoline_imports = {
    'WGTIMUS2' : 'US Gasoline Imports (kbd)',
    'WGTIM_R10-Z00_2' : 'P1 Gasoline Imports (kbd)',
    'WGTIM_R20-Z00_2' : 'P2 Gasoline Imports (kbd)',
    'WGTIM_R30-Z00_2' : 'P3 Gasoline Imports (kbd)',
    'WGTIM_R40-Z00_2' : 'P4 Gasoline Imports (kbd)',
    'WGTIM_R50-Z00_2' : 'P5 Gasoline Imports (kbd)',    
}

gasoline_production = {
    'WGFRPUS2' : 'US Gasoline Production (kbd)',
    'WGFRPP12' : 'P1 Gasoline Production (kbd)',
    'WGFRPP22' : 'P2 Gasoline Production (kbd)',
    'WGFRPP32' : 'P3 Gasoline Production (kbd)',
    'WGFRPP42' : 'P4 Gasoline Production (kbd)',
    'WGFRPP52' : 'P5 Gasoline Production (kbd)',    
}

gasoline_exports = {
    'W_EPM0F_EEX_NUS-Z00_MBBLD' : 'US Gasoline Exports (kbd)',
}

distillate_stocks = {
    'WDISTUS1' : 'US Distillate Stocks (kb)',
    'WDISTP11' : 'P1 Distillate Stocks (kb)',
    'WDISTP21' : 'P2 Distillate Stocks (kb)',
    'WDISTP31' : 'P3 Distillate Stocks (kb)',
    'WDISTP41' : 'P4 Distillate Stocks (kb)',
    'WDISTP51' : 'P5 Distillate Stocks (kb)',
}

distillate_imports = {
    'WDIIMUS2' : 'US Distillate Imports (kbd)',
    'WDIIM_R10-Z00_2' : 'P1 Distillate Imports (kbd)',
    'WDIIM_R20-Z00_2' : 'P2 Distillate Imports (kbd)',
    'WDIIM_R30-Z00_2' : 'P3 Distillate Imports (kbd)',
    'WDIIM_R40-Z00_2' : 'P4 Distillate Imports (kbd)',
    'WDIIM_R50-Z00_2' : 'P5 Distillate Imports (kbd)',    
}

distillate_production = {
    'WDIRPUS2' : 'US Distillate Production (kbd)',
    'WDIRPP12' : 'P1 Distillate Production (kbd)',
    'WDIRPP22' : 'P2 Distillate Production (kbd)',
    'WDIRPP32' : 'P3 Distillate Production (kbd)',
    'WDIRPP42' : 'P4 Distillate Production (kbd)',
    'WDIRPP52' : 'P5 Distillate Production (kbd)',
}

distillate_exports = {
    'WDIEXUS2' : 'US Distillate Exports (kbd)',    
}    

jet_stocks = {
    'WKJSTUS1' : 'US Jet Stocks (kb)',
    'WKJSTP11' : 'P1 Jet Stocks (kb)',
    'WKJSTP21' : 'P2 Jet Stocks (kb)',
    'WKJSTP31' : 'P3 Jet Stocks (kb)',
    'WKJSTP41' : 'P4 Jet Stocks (kb)',
    'WKJSTP51' : 'P5 Jet Stocks (kb)',    
}

jet_imports = {
    'WKJIMUS2' : 'US Jet Imports (kbd)',
    'WKJIM_R10-Z00_2' : 'P1 Jet Imports (kbd)',
    'WKJIM_R20-Z00_2' : 'P2 Jet Imports (kbd)',
    'WKJIM_R30-Z00_2' : 'P3 Jet Imports (kbd)',
    'WKJIM_R40-Z00_2' : 'P4 Jet Imports (kbd)',
    'WKJIM_R50-Z00_2' : 'P5 Jet Imports (kbd)',    
}

jet_production = {
    'WKJRPUS2' : 'US Jet Production (kbd)',
    'WKJRPP12' : 'P1 JetProduction (kbd)',
    'WKJRPP22' : 'P2 Jet Production (kbd)',
    'WKJRPP32' : 'P3 Jet Production (kbd)',
    'WKJRPP42' : 'P4 Jet Production (kbd)',
    'WKJRPP52' : 'P5 Jet Production (kbd)',    
}

jet_exports = {
    'WKJEXUS2' : 'US Jet Exports (kbd)',    
}

fueloil_stocks = {
    'WRESTUS1' : 'US Fuel Oil Stocks (kb)',
    'WRESTP11' : 'P1 Fuel Oil Stocks (kb)',
    'WRESTP21' : 'P2 Fuel Oil Stocks (kb)',
    'WRESTP31' : 'P3 Fuel Oil Stocks (kb)',
    'WRESTP41' : 'P4 Fuel Oil Stocks (kb)',
    'WRESTP51' : 'P5 Fuel Oil Stocks (kb)',    
}

fueloil_imports = {
    'WREIMUS2' : 'US Fuel Oil Imports (kbd)',
    'WREIM_R10-Z00_2' : 'P1 Fuel Oil Imports (kbd)',
    'WREIM_R20-Z00_2' : 'P2 Fuel Oil Imports (kbd)',
    'WREIM_R30-Z00_2' : 'P3 Fuel Oil Imports (kbd)',
    'WREIM_R40-Z00_2' : 'P4 Fuel Oil Imports (kbd)',
    'WREIM_R50-Z00_2' : 'P5 Fuel Oil Imports (kbd)',
}

fueloil_production = {
    'WRERPUS2' : 'US Fuel Oil Production (kbd)',
    'WRERPP12' : 'P1 Fuel Oil Production (kbd)',
    'WRERPP22' : 'P2 Fuel Oil Production (kbd)',
    'WRERPP32' : 'P3 Fuel Oil Production (kbd)',
    'WRERPP42' : 'P4 Fuel Oil Production (kbd)',
    'WRERPP52' : 'P5 Fuel Oil Production (kbd)',    
}

fueloil_exports = {
    'WREEXUS2' : 'US Fuel Oil Exports (kbd)',        
}

propane_propylene_stocks = {
    'WPRSTUS1' : 'US C3/C3= Stocks (kb)',
    'WPRSTP11' : 'P1 C3/C3= Stocks (kb)',
    'WPRSTP21' : 'P2 C3/C3= Stocks (kb)',
    'WPRSTP31' : 'P3 C3/C3= Stocks (kb)',
    'WPRST_R4N5_1' : 'P4P5 C3/C3= Stocks (kb)',    
}

propane_propylene_imports = {
    'WPRIM_NUS-Z00_2' : 'US C3/C3= Imports (kbd)',
    'WPRIMP12' : 'P1 C3/C3= Imports (kbd)',
    'WPRIMP22' : 'P2 C3/C3= Imports (kbd)',
    'WPRIMP32' : 'P3 C3/C3= Imports (kbd)',
    'W_EPLLPZ_IM0_R45-Z00_MBBLD' : 'P4P5 C3/C3= Imports (kbd)',    
}

propane_propylene_production = {
    'WPRTP_NUS_2' : 'US C3/C3= Production (kbd)',
    'WPRNPP12' : 'P1 C3/C3= Production (kbd)',
    'WPRNPP22' : 'P2 C3/C3= Production (kbd)',
    'WPRNPP32' : 'P3 C3/C3= Production (kbd)',
    'W_EPLLPZ_YPT_R4N5_MBBLD' : 'P4P5 C3/C3= Production (kbd)',    
}

propane_propylene_exports = {
    'W_EPLLPZ_EEX_NUS-Z00_MBBLD' : 'US C3/C3= Exports (kbd)',    
}


In [6]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.Div([
        # First Column
        html.Div([
            html.H1('Headline', className='eia_h1_header'),
            generate_dash_table(df, headline, 'Headline', '100%'),
            generate_dash_table(df, products_supplied, 'Products Supplied', '100%'),
        ], className='eia_table_style'),
        
        # Second Column
        html.Div([
            html.H1('Crude', className='eia_h1_header'),
            generate_dash_table(df, crude_stocks, 'Crude Stocks', '100%'),
            generate_dash_table(df, crude_other_stocks, 'Crude Other Stocks', '100%'),
            generate_dash_table(df, crude_production, 'Crude Production', '100%'),
            generate_dash_table(df, crude_imports, 'Crude Imports', '100%'),
            generate_dash_table(df, crude_adjustment, 'Crude Adjustment', '100%'),
            generate_dash_table(df, crude_runs, 'Crude Runs', '100%'),
            generate_dash_table(df, crude_exports, 'Crude Exports', '100%'),
        ], className='eia_table_style'),

        html.Div([
            html.H1('Gasoline', className='eia_h1_header'),
            generate_dash_table(df, gasoline_stocks, 'Gasoline Stocks', '100%'),
            generate_dash_table(df, gasoline_imports, 'Gasoline Imports', '100%'),
            generate_dash_table(df, gasoline_production, 'Gasoline Production', '100%'),
            generate_dash_table(df, gasoline_exports, 'Gasoline Exports', '100%'),
        ], className='eia_table_style'),        

        html.Div([
            html.H1('Distillate', className='eia_h1_header'),
            generate_dash_table(df, distillate_stocks, 'Distillate Stocks', '100%'),
            generate_dash_table(df, distillate_imports, 'Distillate Imports', '100%'),
            generate_dash_table(df, distillate_production, 'Distillate Production', '100%'),
            generate_dash_table(df, distillate_exports, 'Distillate Exports', '100%'),
        ], className='eia_table_style'),        

        html.Div([
            html.H1('Jet', className='eia_h1_header'),
            generate_dash_table(df, jet_stocks, 'Jet Stocks', '100%'),
            generate_dash_table(df, jet_imports, 'Jet Imports', '100%'),
            generate_dash_table(df, jet_production, 'Jet Production', '100%'),
            generate_dash_table(df, jet_exports, 'Jet Exports', '100%'),
        ], className='eia_table_style'),        
        
        html.Div([
            html.H1('Fuel Oil', className='eia_h1_header'),
            generate_dash_table(df, fueloil_stocks, 'Fuel Oil Stocks', '100%'),
            generate_dash_table(df, fueloil_imports, 'Fuel Oil Imports', '100%'),
            generate_dash_table(df, fueloil_production, 'Fuel Oil Production', '100%'),
            generate_dash_table(df, fueloil_exports, 'Fuel Oil Exports', '100%'),
        ], className='eia_table_style'),  
        
        html.Div([
            html.H1('Propane/Propylene', className='eia_h1_header'),
            generate_dash_table(df, propane_propylene_stocks, 'Propane/Propylene Stocks', '100%'),
            generate_dash_table(df, propane_propylene_imports, 'Propane/Propylene Imports', '100%'),
            generate_dash_table(df, propane_propylene_production, 'Propane/Propylene Production', '100%'),
            generate_dash_table(df, propane_propylene_exports, 'Propane/Propylene Exports', '100%'),
        ], className='eia_table_style'),

    ], style={'display': 'flex', 'width': '3000px'}),
    
], style={'display': 'flex', 'justify-content': 'left'})

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)